In [52]:
from itertools import permutations, repeat
import nltk
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet as wn
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from gensim.models import word2vec

1. Modelos bag of words
2. Pre-processamento com lowercasing, stemming e remoção de caracteres não alfa-numericos

In [2]:
def correct_text(s):
    
    if isinstance(s, str):
        s = s.lower()
        s = s.replace(".", ". ")
        for s1 in range(0, 10):
            s = s.replace(". " + str(s1), "." + str(s1))
        
    
    s = s.lower()
    s = s.replace(" in.","in.")
    s = s.replace(" inch","in.")
    s = s.replace("inch","in.")
    s = s.replace(" in ","in. ")
    s = s.replace(" ft ","ft. ")
    s = s.replace(" ft.","ft.")
    s = s.replace(" foot","ft.")
    s = s.replace(" feet","ft.")
    s = s.replace("foot","ft.")
    s = s.replace("feet","ft.")
    s = s.replace(" ft ","ft. ")
    s = s.replace(" gallon ","gal. ")
    s = s.replace("gallon","gal.")
    s = s.replace(" oz.","oz.")
    s = s.replace(" ounce","oz.")
    s = s.replace("ounce","oz.")
    s = s.replace(" oz ","oz. ")
    s = s.replace(" cm.","cm.")
    s = s.replace(" cm ","cm. ")
    s = s.replace('H x', 'height')
    s = s.replace('sq.', 'square')
    s = s.replace('cu.', 'cubic')
    s = s.replace('lbs.', 'pounds')
    s = s.replace('W x', 'width')
    s = s.replace('H x', 'height')
    s = s.replace('Ah ', 'amphere')
    
    return s

def clean_text(text):
    text = correct_text(text)
    # Remove '&nbsp;' from the text content before HTML tags strip off.
    text.replace('&nbsp;', ' ')
    # Remove HTML tags
    text = BeautifulSoup(text, "lxml").get_text(separator=" ")
    # Replace all punctuation and special characters by space
    text.replace("[ &<>)(_,.;:!?/-]+", " ")
    # Remove the apostrophe's
    text.replace("'s\\b", "")
    # Remove the apostrophe
    text.replace("[']+", "")
    # Remove the double quotes
    text.replace("[\"]+", "")
    # Convert to lower case, split into individual words
    words = text.lower().split()
    return( " ".join( words ))
 

In [3]:
train_data = pd.read_csv('../data/train.csv', encoding="ISO-8859-1")
test_data = pd.read_csv('../data/test.csv', encoding="ISO-8859-1")
attribute_data = pd.read_csv('../data/attributes.csv')
descriptions = pd.read_csv('../data/product_descriptions.csv')

train_data = pd.merge(train_data, descriptions, on="product_uid", how="left")
test_data = pd.merge(test_data, descriptions, on="product_uid", how="left")

product_count = pd.DataFrame(pd.Series(train_data.groupby(["product_uid"]).size(), name="product_count"))
product_count = pd.DataFrame(pd.Series(test_data.groupby(["product_uid"]).size(), name="product_count"))

In [4]:
english_sw = stopwords.words('english')

In [5]:
def clean(sentence):
    
    tokenizer = RegexpTokenizer(r'\w+')
    stemmer = PorterStemmer()
    tokens = tokenizer.tokenize(sentence.lower())
    tokens = [token for token in tokens if token not in english_sw]
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

In [6]:
train_data['product_description'] = train_data.apply(lambda x: clean_text(x['product_description']),axis=1)
train_data['product_title'] = train_data.apply(lambda x: clean_text(x['product_title']),axis=1)
train_data['search_term'] = train_data.apply(lambda x: clean_text(x['search_term']),axis=1)

In [ ]:
test_data['product_description'] = test_data.apply(lambda x: clean_text(x['product_description']),axis=1)
test_data['product_title'] = test_data.apply(lambda x: clean_text(x['product_title']),axis=1)
test_data['search_term'] = test_data.apply(lambda x: clean_text(x['search_term']),axis=1)

In [7]:
train_data['description_tokens'] = train_data.apply(lambda x: word_tokenize(x['product_description']), axis=1)
train_data['description_tokens_clean'] = train_data.apply(lambda x: clean(x['product_description']), axis=1)

In [ ]:
test_data['description_tokens'] = test_data.apply(lambda x: word_tokenize(x['product_description']), axis=1)
test_data['description_tokens_clean'] = test_data.apply(lambda x: clean(x['product_description']), axis=1)

In [8]:
train_data['title_tokens'] = train_data.apply(lambda x: word_tokenize(x['product_title']), axis=1)
train_data['title_tokens_clean'] = train_data.apply(lambda x: clean(x['product_title']), axis=1)

In [ ]:
test_data['title_tokens'] = test_data.apply(lambda x: word_tokenize(x['product_title']), axis=1)
test_data['title_tokens_clean'] = test_data.apply(lambda x: clean(x['product_title']), axis=1)

In [9]:
train_data['search_tokens'] = train_data.apply(lambda x: word_tokenize(x['search_term']), axis=1)
train_data['search_tokens_clean'] = train_data.apply(lambda x: clean(x['search_term']), axis=1)

In [ ]:
test_data['search_tokens'] = test_data.apply(lambda x: word_tokenize(x['search_term']), axis=1)
test_data['search_tokens_clean'] = test_data.apply(lambda x: clean(x['search_term']), axis=1)

In [10]:
train_data['n_tokens_desc'] = train_data.apply(lambda x: len(x['description_tokens']), axis=1)
train_data['n_tokens_title'] = train_data.apply(lambda x: len(x['title_tokens']), axis=1)
train_data['n_tokens_search'] = train_data.apply(lambda x: len(x['search_tokens']), axis=1)

In [ ]:
test_data['n_tokens_desc'] = test_data.apply(lambda x: len(x['description_tokens']), axis=1)
test_data['n_tokens_title'] = test_data.apply(lambda x: len(x['title_tokens']), axis=1)
test_data['n_tokens_search'] = test_data.apply(lambda x: len(x['search_tokens']), axis=1)

In [11]:
train_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,description_tokens,description_tokens_clean,title_tokens,title_tokens_clean,search_tokens,search_tokens_clean,n_tokens_desc,n_tokens_title,n_tokens_search
0,2,100001,simpson strong-tie 12-gauge angle,angle bracket,3.00,"not only do angles make joints stronger, they ...","[not, only, do, angles, make, joints, stronger...","[angl, make, joint, stronger, also, provid, co...","[simpson, strong-tie, 12-gauge, angle]","[simpson, strong, tie, 12, gaug, angl]","[angle, bracket]","[angl, bracket]",144,4,2
1,3,100001,simpson strong-tie 12-gauge angle,l bracket,2.50,"not only do angles make joints stronger, they ...","[not, only, do, angles, make, joints, stronger...","[angl, make, joint, stronger, also, provid, co...","[simpson, strong-tie, 12-gauge, angle]","[simpson, strong, tie, 12, gaug, angl]","[l, bracket]","[l, bracket]",144,4,2
2,9,100002,behr premium textured deckover 1-gal. #sc-141 ...,deck over,3.00,behr premium textured deckover is an innovativ...,"[behr, premium, textured, deckover, is, an, in...","[behr, premium, textur, deckov, innov, solid, ...","[behr, premium, textured, deckover, 1-gal, ., ...","[behr, premium, textur, deckov, 1, gal, sc, 14...","[deck, over]",[deck],190,13,2
3,16,100005,delta vero 1-handle shower only faucet trim ki...,rain shower head,2.33,update your bathroom with the delta vero singl...,"[update, your, bathroom, with, the, delta, ver...","[updat, bathroom, delta, vero, singl, handl, s...","[delta, vero, 1-handle, shower, only, faucet, ...","[delta, vero, 1, handl, shower, faucet, trim, ...","[rain, shower, head]","[rain, shower, head]",113,15,3
4,17,100005,delta vero 1-handle shower only faucet trim ki...,shower only faucet,2.67,update your bathroom with the delta vero singl...,"[update, your, bathroom, with, the, delta, ver...","[updat, bathroom, delta, vero, singl, handl, s...","[delta, vero, 1-handle, shower, only, faucet, ...","[delta, vero, 1, handl, shower, faucet, trim, ...","[shower, only, faucet]","[shower, faucet]",113,15,3


In [12]:
#train_data['product_description']

### Bag-of-Word Features

1. feature_1 = is search term a substring in title?
2. feature_2 = is search term a substring in description?
3. feature_3 = proportion of terms of search in title - no stemm, no stopword removal
4. feature_4 = proportion of terms of search in title - stemm and stopword removal
5. feature_5 = proportion of terms of search in description - no stemm, no stopword removal
6. feature_6 = proportion of terms of search in description - stemm and stopword removal
7. feature_7 = length of search
8. feature_8 = length of description
9. feature_9 = length of title
7. features_10 to 43 = if word i in search is in description
7. features_44 to 77 = if word i in search is in title

#### Ideias

1. 1 feature para cada palavra de busca indicando se ela está no titulo ou descrição
2. Usando word2vec, distance entre palavras da busca e do titulo/descrição
3. Usando topic modeling: Probabilidade de busca conter o mesmo topico que titulo/descrição

In [12]:
train_data['feature_1'] = train_data.apply(lambda x: int(x['search_term'] in x['product_title']), axis=1)
train_data['feature_2'] = train_data.apply(lambda x: int(x['search_term'] in x['product_description']), axis=1)
train_data['feature_3'] = train_data.apply(lambda x: float(len( set(x['search_tokens']).intersection(set(x['title_tokens']))))/len( set(x['search_tokens'])) , axis=1)
train_data['feature_4'] = train_data.apply(lambda x: float(len( set(x['search_tokens_clean']).intersection(set(x['title_tokens_clean']))))/len( set(x['search_tokens'])) , axis=1)
train_data['feature_5'] = train_data.apply(lambda x: float(len( set(x['search_tokens']).intersection(set(x['description_tokens']))))/len( set(x['search_tokens'])) , axis=1)
train_data['feature_6'] = train_data.apply(lambda x: float(len( set(x['search_tokens_clean']).intersection(set(x['description_tokens_clean']))))/len( set(x['search_tokens'])) , axis=1)
train_data['feature_7'] = train_data.apply(lambda x: float(len(x['search_tokens'])), axis=1)
train_data['feature_8'] = train_data.apply(lambda x: float(len(x['description_tokens'])), axis=1)
train_data['feature_9'] = train_data.apply(lambda x: float(len(x['title_tokens'])), axis=1)

In [ ]:
test_data['feature_1'] = test_data.apply(lambda x: int(x['search_term'] in x['product_title']), axis=1)
test_data['feature_2'] = test_data.apply(lambda x: int(x['search_term'] in x['product_description']), axis=1)
test_data['feature_3'] = test_data.apply(lambda x: float(len( set(x['search_tokens']).intersection(set(x['title_tokens']))))/len( set(x['search_tokens'])) , axis=1)
test_data['feature_4'] = test_data.apply(lambda x: float(len( set(x['search_tokens_clean']).intersection(set(x['title_tokens_clean']))))/len( set(x['search_tokens'])) , axis=1)
test_data['feature_5'] = test_data.apply(lambda x: float(len( set(x['search_tokens']).intersection(set(x['description_tokens']))))/len( set(x['search_tokens'])) , axis=1)
test_data['feature_6'] = test_data.apply(lambda x: float(len( set(x['search_tokens_clean']).intersection(set(x['description_tokens_clean']))))/len( set(x['search_tokens'])) , axis=1)
test_data['feature_7'] = test_data.apply(lambda x: float(len(x['search_tokens'])), axis=1)
test_data['feature_8'] = test_data.apply(lambda x: float(len(x['description_tokens'])), axis=1)
test_data['feature_9'] = test_data.apply(lambda x: float(len(x['title_tokens'])), axis=1)

In [13]:
train_data.head(3)

,id,product_uid,product_title,search_term,relevance,product_description,description_tokens,description_tokens_clean,title_tokens,title_tokens_clean,...,n_tokens_search,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,2,100001,simpson strong-tie 12-gauge angle,angle bracket,3.0,"not only do angles make joints stronger, they ...","[not, only, do, angles, make, joints, stronger...","[angl, make, joint, stronger, also, provid, co...","[simpson, strong-tie, 12-gauge, angle]","[simpson, strong, tie, 12, gaug, angl]",...,2,0,0,0.5,0.5,0.0,0.5,2,144,4
1,3,100001,simpson strong-tie 12-gauge angle,l bracket,2.5,"not only do angles make joints stronger, they ...","[not, only, do, angles, make, joints, stronger...","[angl, make, joint, stronger, also, provid, co...","[simpson, strong-tie, 12-gauge, angle]","[simpson, strong, tie, 12, gaug, angl]",...,2,0,0,0.0,0.0,0.0,0.0,2,144,4
2,9,100002,behr premium textured deckover 1-gal. #sc-141 ...,deck over,3.0,behr premium textured deckover is an innovativ...,"[behr, premium, textured, deckover, is, an, in...","[behr, premium, textur, deckov, innov, solid, ...","[behr, premium, textured, deckover, 1-gal, ., ...","[behr, premium, textur, deckov, 1, gal, sc, 14...",...,2,0,0,0.0,0.0,0.5,0.5,2,190,13


In [14]:
max([len(x) for x in train_data['search_tokens']])

14

In [ ]:
max([len(x) for x in test_data['search_tokens']])

In [15]:
def word_search(word_list, index, tokens):
    if len(word_list) < index + 1:
        return 0
    text = ''.join(tokens)
    return int(word_list[index] in text)

In [16]:
for index in range(0,14):
    train_data['feature_{}'.format(index+10)] = train_data.apply(lambda x: word_search(x['search_tokens'], index, x['title_tokens']), axis=1)
    test_data['feature_{}'.format(index+10)] = test_data.apply(lambda x: word_search(x['search_tokens'], index, x['title_tokens']), axis=1)

In [17]:
for index in range(0,14):
    train_data['feature_{}'.format(index+24)] = train_data.apply(lambda x: word_search(x['search_tokens'], index, x['description_tokens']), axis=1)
    test_data['feature_{}'.format(index+24)] = test_data.apply(lambda x: word_search(x['search_tokens'], index, x['description_tokens']), axis=1)

In [92]:
train_data.columns

Index([                      u'id',              u'product_uid',
                  u'product_title',              u'search_term',
                      u'relevance',      u'product_description',
             u'description_tokens', u'description_tokens_clean',
                   u'title_tokens',       u'title_tokens_clean',
                  u'search_tokens',      u'search_tokens_clean',
                  u'n_tokens_desc',           u'n_tokens_title',
                u'n_tokens_search',                u'feature_1',
                      u'feature_2',                u'feature_3',
                      u'feature_4',                u'feature_5',
                      u'feature_6',                u'feature_7',
                      u'feature_8',                u'feature_9',
                     u'feature_10',               u'feature_11',
                     u'feature_12',               u'feature_13',
                     u'feature_14',               u'feature_15',
                     u'fe

In [ ]:
bow_features_train = train_data[['id', 'relevance'] + ['feature_{0}'.format(i) for i in range(1,38)]]
bow_features_test = test_data[['id'] + ['feature_{0}'.format(i) for i in range(1,38)]]

In [ ]:
bow_features_train.to_csv('../data/bow_features.csv', index=False)
bow_features_test.to_csv('../data/bow_features_test.csv', index=False)

### Word2Vec Features

1. feature_1 = is search term a substring in title?
2. feature_2 = is search term a substring in description?
3. feature_3 = proportion of terms of search in title - no stemm, no stopword removal
4. feature_4 = proportion of terms of search in title - stemm and stopword removal
5. feature_5 = proportion of terms of search in description - no stemm, no stopword removal
6. feature_6 = proportion of terms of search in description - stemm and stopword removal
7. feature_7 = length of search
8. feature_8 = length of description
9. feature_9 = length of title
7. features_10 to 23 = lowest similarity of word i in search and the words in the description
7. features_24 to 37 = lowest similarity of word i in search and the words in the title

#### Ideias

1. 1 feature para cada palavra de busca indicando se ela está no titulo ou descrição
2. Usando word2vec, distance entre palavras da busca e do titulo/descrição
3. Usando topic modeling: Probabilidade de busca conter o mesmo topico que titulo/descrição

In [19]:
descriptions = list(train_data.product_description)
title = list(train_data.product_title)


In [20]:
sentences = descriptions + title

In [21]:
sentences = [[word for word in sentence.lower().split()] for sentence in sentences]

In [22]:
from collections import defaultdict

frequency = defaultdict(int)
for sentence in sentences:
    for token in sentence:
        frequency[token] += 1
        
sentences = [[token for token in sentence if frequency[token] > 1] for sentence in sentences]


In [23]:
model = word2vec.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

In [24]:
model.vocab

{u'fawn': <gensim.models.word2vec.Vocab at 0x7f19c1b1f890>,
 u'wooda': <gensim.models.word2vec.Vocab at 0x7f1935800890>,
 u'double-pole,': <gensim.models.word2vec.Vocab at 0x7f1935800950>,
 u'nano-polymer': <gensim.models.word2vec.Vocab at 0x7f1935800990>,
 u'raining': <gensim.models.word2vec.Vocab at 0x7f19343b9c50>,
 u'1,800': <gensim.models.word2vec.Vocab at 0x7f1935800910>,
 u'wrought-iron': <gensim.models.word2vec.Vocab at 0x7f19343b9c90>,
 u'polypropylenelatex': <gensim.models.word2vec.Vocab at 0x7f19343b9c10>,
 u'woods': <gensim.models.word2vec.Vocab at 0x7f1935800a50>,
 u'spiders': <gensim.models.word2vec.Vocab at 0x7f1935800a90>,
 u'hanging': <gensim.models.word2vec.Vocab at 0x7f1935800ad0>,
 u'outagesoxygen': <gensim.models.word2vec.Vocab at 0x7f1935800b10>,
 u'cellulose/cellulene': <gensim.models.word2vec.Vocab at 0x7f19343b9cd0>,
 u'localized': <gensim.models.word2vec.Vocab at 0x7f1935800bd0>,
 u'11in.suitable': <gensim.models.word2vec.Vocab at 0x7f1933dd9410>,
 u'canes': <

In [43]:
query_terms = set(itertools.chain(*train_data.search_tokens_clean))
title_terms = set(itertools.chain(*train_data.title_tokens_clean))
description_terms = set(itertools.chain(*train_data.description_tokens_clean))

In [4]:
def lowest_similarity(search_token_list, attribute_token_list, index, model):
    
    if len(search_token_list) < index + 1:
        return 0.
    else:
        search_token = search_token_list[index]
    
    similarities = []
    tuples = [[search_token, attribute_token]  for attribute_token in attribute_token_list]
    
    for x,y in tuples:
        try:
            similarities.append(model.similarity(x,y))
        except:
            continue
            
    if not similarities:
        return 0.
    
    return min(similarities)


In [97]:
lowest_similarity(['angl', 'bracket'], ['simpson', 'strong', 'tie', '12', 'gaug', 'angl'], model)

0.0094482086301302792

In [98]:
for index in range(0,14):
    train_data['feature_{}'.format(index+10)] = train_data.apply(lambda x: lowest_similarity(x['search_tokens'], x['title_tokens'], index, model), axis=1)
    test_data['feature_{}'.format(index+10)] = test_data.apply(lambda x: lowest_similarity(x['search_tokens'], x['title_tokens'], index, model), axis=1)

In [99]:
for index in range(0,14):
    train_data['feature_{}'.format(index+24)] = train_data.apply(lambda x: lowest_similarity(x['search_tokens'], x['description_tokens'], index, model), axis=1)
    test_data['feature_{}'.format(index+24)] = test_data.apply(lambda x: lowest_similarity(x['search_tokens'], x['description_tokens'], index, model), axis=1)

0        0.009448
1       -0.109302
2       -0.024332
3       -0.010829
4       -0.110568
5       -0.095378
6       -0.050438
7        0.000000
8        0.000314
9       -0.079166
10      -0.091999
11      -0.147929
12      -0.126832
13      -0.147929
14      -0.093258
15       0.000000
16      -0.056631
17      -0.042951
18      -0.120730
19      -0.115110
20       0.035665
21      -0.060299
22      -0.054350
23      -0.001439
24      -0.038161
25      -0.038242
26      -0.001439
27      -0.086624
28      -0.015021
29      -0.144477
           ...   
74037   -0.108048
74038   -0.277745
74039   -0.052302
74040   -0.178715
74041   -0.190207
74042   -0.004818
74043   -0.117944
74044    0.000000
74045   -0.138871
74046   -0.143931
74047   -0.133114
74048   -0.204294
74049   -0.138811
74050    0.033196
74051   -0.180118
74052    0.022774
74053   -0.117025
74054    0.032394
74055   -0.144439
74056   -0.020438
74057   -0.168628
74058    0.046351
74059   -0.079801
74060    0.021694
74061   -0

In [101]:
w2v_features_train = train_data[['id', 'relevance'] + ['feature_{0}'.format(i) for i in range(1,38)]]
w2v_features_test = test_data[['id'] + ['feature_{0}'.format(i) for i in range(1,38)]]

In [130]:
w2v_features_train.to_csv('../data/w2v_features.csv', index=False)
w2v_features_test.to_csv('../data/w2v_features_test.csv', index=False)